In [1]:
import numpy as np
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import h5py

import keras
import keras.layers as KL
import keras.models as KM

import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

from data_utils.hdf5_sequence import HDF5Sequence

Using TensorFlow backend.



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



## Constants

In [2]:
MODEL_NAME = 'bi_rnn_test'
MODEL_SAVE_PATH = './models/checkpoints/{}/'.format(MODEL_NAME)

PRETRAINED_MODEL_NAME = 'hierdec-trio_16bar'
config = configs.CONFIG_MAP[PRETRAINED_MODEL_NAME]

DATA_PATH = './data/lmd_clean/'
HDF5_PATH = DATA_PATH + 'lmd_clean_split_z.h5'

TIMESTEPS = 256
DIM_MELODY = 90
DIM_LATENT = 512

In [3]:
config.hparams

HParams([('batch_size', 256), ('beta_rate', 0.0), ('clip_mode', 'global_norm'), ('conditional', True), ('dec_rnn_size', [1024, 1024]), ('decay_rate', 0.9999), ('dropout_keep_prob', 1.0), ('enc_rnn_size', [2048, 2048]), ('free_bits', 256), ('grad_clip', 1.0), ('grad_norm_clip_to_zero', 10000), ('learning_rate', 0.001), ('max_beta', 0.2), ('max_seq_len', 256), ('min_learning_rate', 1e-05), ('residual_decoder', False), ('residual_encoder', False), ('sampling_rate', 0.0), ('sampling_schedule', 'constant'), ('use_cudnn', False), ('z_size', 512)])

## Hyperparameters

In [4]:
batch_size = 32

## Data pipeline

In [5]:
data_file = h5py.File(HDF5_PATH, 'r')

In [6]:
train_seq = HDF5Sequence(data_file, batch_size, 
                         index_path=DATA_PATH + 'train_indices.csv')
val_seq = HDF5Sequence(data_file, batch_size, 
                       index_path=DATA_PATH + 'val_indices.csv')

## Define model: bidirectional RNN

In [7]:
input_layer = KL.Input(shape=(TIMESTEPS, DIM_MELODY), name='input')
layer = KL.Bidirectional(
    KL.CuDNNLSTM(128, return_sequences=True, name='bi_lstm_1'))(input_layer)
layer = KL.Bidirectional(KL.CuDNNLSTM(128, name='bi_lstm_2'))(layer)
output_layer = KL.Dense(DIM_LATENT, activation='linear', name='output')(layer)

model = KM.Model(inputs=input_layer, outputs=output_layer)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 256, 90)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256, 256)          224256    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               394240    
_________________________________________________________________
output (Dense)               (None, 512)               131584    
Total params: 750,080
Trainable params: 750,080
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(optimizer=keras.optimizers.rmsprop(lr=1e-3),
              loss='mean_squared_error')

In [9]:
model.fit_generator(train_seq, steps_per_epoch=len(train_seq),
                    validation_data=val_seq, validation_steps=len(val_seq),
                    max_queue_size=128, workers=32)

Epoch 1/1
 281/1593 [====>.........................] - ETA: 25:54 - loss: 0.8918

KeyboardInterrupt: 